In [54]:
import numpy as np
import pandas as pd
import re

import praw
import emoji
from concurrent.futures import ThreadPoolExecutor, as_completed

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
COMMON_WORDS = set(stopwords.words("english"))

In [90]:
class Get_subreddit():
    def __init__(self, subreddit):
        reddit = praw.Reddit(
            client_id="BBO_q72yVclp5g",
            client_secret="ZRNH4nLoNBo8gI97pPZMydhEzwMj3A",
            user_agent="hsabbar",
        )
        self.hot = reddit.subreddit(subreddit).hot(limit=1)
        self.max_comments = 100
        self.data = []
        self.columns = ['title','id','score', 'date', 'author', 'content', 'num_comments', 'comments'] 
        
    def get_data(self, submission):
        data_tmp = []
        comment_data = []
        data_tmp = [submission.title, submission.id, submission.score,submission.created_utc, submission.author, 
                     submission.selftext, len(submission.comments)]

        submission.comments.replace_more(limit=self.max_comments)
        comment_data = [top_level_comment.body for top_level_comment in submission.comments[:self.max_comments]]
        

        """
        for top_level_comment in submission.comments[:self.max_comments]: 
            comment_data.append(top_level_comment.body)
        if len(comment_data) < self.max_comments:
            comment_data = comment_data + [None] * (self.max_comments - len(comment_data))
        """
        return data_tmp + [' '.join(comment_data)]
    
    def get_data_multi_thread(self):
        processes = []
        with ThreadPoolExecutor(max_workers=62) as executor:
            for submission in self.hot:
                processes.append(executor.submit(self.get_data, submission))

        self.data = [task.result() for task in as_completed(processes)]
        
        #print(self.data)
        return pd.DataFrame(self.data, columns = self.columns)

In [101]:
%%time
reddit_data = Get_subreddit("Bitcoin")
df = reddit_data.get_data_multi_thread()
df.head()

Wall time: 768 ms


,title,id,score,date,author,content,num_comments,comments
0,"⚡ Lightning Thursday! May 27, 2021: Explore th...",nm0wks,38,1.622095e+09,rBitcoinMod,The lightning network is a second-layer soluti...,7,What's the simplest video resource to have ln ...


In [102]:
class Data_processing():
    def __init__(self, df_o):
        self.df = df_o
        self.columns_text = ['title', 'content', 'comments'] #+ [f'comment{i}' for i in range(1, 101)]
        self.del_url = re.compile(r'https(.*?)(\s|$)')
    
    
    def my_stopwords(self, word):
        if not word in COMMON_WORDS:
            return word
        else:
            pass

    def multi_stopwords(self, text):
        processes = []
        text_tokens = word_tokenize(text)
        print(text_tokens)
        with ThreadPoolExecutor(max_workers=62) as executor:
            for word in text_tokens:
                processes.append(executor.submit(self.my_stopwords, word))
        
        data = [task.result() for task in as_completed(processes)]
        print(type(data), data)
        #return " ".join(data)
        
    def nlp_pipeline(self, text):
        if text is not None :
            text = text.lower()
            text = text.replace('\n', ' ').replace('\r', '')
            text = ' '.join(text.split())
            text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", " ", text)
            text = re.sub(r"(\s\-\s|-$)", "", text)
            #text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
            text = re.sub(r"\&\S*\s", " ", text)
            text = re.sub(r"['&@¨=“”’‘+—^.;:-_~!\{\}*=\%?,#<>\"()\[\]]", " ", text)
            
            for url in self.del_url.findall(text):
                text = re.sub(f'https{url[0]}', " ", text)
            text = emoji.get_emoji_regexp().sub(" ", text) 
            
            text = self.multi_stopwords(text)
            return text
        else :
            pass
    
    def clean_data(self):
        for col_txt in self.columns_text :
            self.df[col_txt] = self.df[col_txt].apply(lambda x:  self.nlp_pipeline(x))
        
        return self.df

In [103]:
%%time
data_proc = Data_processing(df)
df2 = data_proc.clean_data()
df2.head()

['lightning', 'thursday', 'may', 'explore', 'the', 'lightning', 'network']
<class 'list'> ['thursday', 'network', None, 'lightning', 'lightning', 'explore', 'may']
['the', 'lightning', 'network', 'is', 'a', 'second-layer', 'solution', 'on', 'top', 'of', 'the', 'bitcoin', 'blockchain', 'that', 'enables', 'quick', 'cheap', 'and', 'scalable', 'bitcoin', 'payments', 'here', 'is', 'the', 'place', 'to', 'discuss', 'and', 'learn', 'more', 'about', 'lightning', 'ask', 'your', 'questions', 'about', 'lightning', 'provide', 'reviews', 'feedback', 'comparisons', 'of', 'ln', 'apps', 'services', 'websites', 'etc', 'learn', 'about', 'new', 'ln', 'features', 'development', 'apps', 'link', 'to', 'good', 'quality', 'resources', 'articles', 'wikis', 'etc', 'resources', 'here', 'is', 'an', 'awesome', 'list', 'of', 'resources', 'compiled', 'by', 'jameson', 'lopp', '//www', 'lopp', 'net/lightning-information', 'html', 'want', 'to', 'test', 'out', 'your', 'lightning', 'fire', 'power', 'tip', 'the', 'bitcoin'

,title,id,score,date,author,content,num_comments,comments
0,None,nm0wks,38,1.622095e+09,rBitcoinMod,None,7,None
